# Emotional Speech Recognition
This notebook will be an audio classification problem and solved with Audio Feature extraction and augmentation, Machine Learning and Deep Learning. 

https://www.kaggle.com/dejolilandry/asvpesdspeech-nonspeech-emotional-utterances

In [9]:
import numpy as np 
import pandas as pd
import os 
import math
import torch 
import torchaudio 
import matplotlib.pyplot as plt 
#%matplotlib_inline
import seaborn as sns
import librosa 
import librosa.display

import IPython.display as ipd

# Data Preperation 
Since the dataset didnt come with a CSV file containing the metadata we'll create a dataframe that contains the Path to the Wav file and it Class which is what emotion it has been labeled as. 

Taken from the Kaggle Dataset Descri

Each wav file in the dataset consists of numerical identifiers. 

Filename identifiers:
Modality ( 03 = audio-only).
Vocal channel on s(01 = speech, 02 = npeech).
Emotion ( 01 = boredom, 02 = neutral, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised, 09 = excited, 10 = pleasure, 11 = pain, 12 = disappointment, 13 = others).
Emotional intensity (01 = normal, 02 = high).
Statement (as it’s non scripted this refer to the number of sample select per actor folder ).
Actor ( even numbered acteurs are male, odd numbered actors are female).
Age(01 = above 65, 02 = between 20~64, 03 = under 20,04=new born).
Source of downloading (01 =website , 02 = youtube channel, 03= movies).
Language(01=Chinese , 02=English ,04 = french , others).



In [53]:
labels_dict = {
    1: 'boredom', 
    2: 'neutral',
    3: 'happy',
    4: 'sad',
    5: 'angry',
    6: 'fearful',
    7: 'disgust', 
    8: 'surprised',
    9: 'excited', 
    10: 'pleasure',
    11: 'pain', 
    12: 'disappointment', 
    13: 'others'
}

So by taking a look at how the data is labeled we can decide how to make classification for our problem. Lets start simple and just use the Emotion label. Later we can try classifying other labels like age and gender. We'll also only use files labeled as speech, later we'll use the non speech files for data augmentation but more on that later. for now lets make a list 

In [50]:
# lets practice taking the path strings apart 
ster = "/Users/stephen/Desktop/Speech_Commands/Data/ASVP-ESD_UPDATE/Audio/actor_103/03-01-05-01-07-103-02-01-02-18.wav"
part = ster.split('/')[-1]
part = part.split('.')[0]
part = part.split('-')
print(part)

['03', '01', '05', '01', '07', '103', '02', '01', '02', '18']


In [56]:
# string representing PATH for folder containing data
audio_path = '/Users/stephen/Desktop/Speech_Commands/Data/ASVP-ESD_UPDATE/Audio/'

# the dataset is organized into folders for each actor/actress. the listdir method will list all contained folders 
dir_list = os.listdir(audio_path)


file_emotion = []
file_path = []

# iterate through files and 
for dir in dir_list:
    # extract contents from each folder 
    actor = os.listdir(audio_path + dir)
    # go through each file in each folder 
    for file in actor: 
        part = file.split('.')[0] # ignore the .wav
        part = part.split('-') 
        # the second part represents speech or non speech 
        if part[1] == '01': # 01 represents Speech 
            # third part represents emotion 
            file_emotion.append(int(part[2]))
            file_path.append(audio_path + dir + '/' + file)



# dataframe for emotion of files 
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files 
path_df = pd.DataFrame(file_path, columns=['Path'])
metadata = pd.concat([emotion_df, path_df], axis=1)

# change integers to actual emotions. 
metadata.Emotions.replace(labels_dict, inplace=True)
metadata.head()

,Emotions,Path
0,neutral,/Users/stephen/Desktop/Speech_Commands/Data/AS...
1,angry,/Users/stephen/Desktop/Speech_Commands/Data/AS...
2,neutral,/Users/stephen/Desktop/Speech_Commands/Data/AS...
3,neutral,/Users/stephen/Desktop/Speech_Commands/Data/AS...
4,angry,/Users/stephen/Desktop/Speech_Commands/Data/AS...


In [62]:
# Sanity check to make sure the Path Column contains the fill path 
metadata['Path'][0]

'/Users/stephen/Desktop/Speech_Commands/Data/ASVP-ESD_UPDATE/Audio/actor_16/03-01-02-01-02-16-03-03-01.wav'

In [63]:
# lets save to CSV file for later use. The PandasData frame will be used to visualize some data 
metadata.to_csv('/Users/stephen/Desktop/Speech_Commands/Data/ASVP-ESD_UPDATE/metadata.csv')